In [43]:
from glob import glob
import constants as cn
import os
import time
import joblib
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
test_dict = {0:'Sick',1:'Healthy'}

test_dict_af = {0:'Alzheimers',1:'Dementia'}

In [45]:
eeg_channels = [
 'F3',
 'F4',
 'F7',
 'F8',
 'T3',
 'T4',
 'T5',
 'T6',
 'Fz']

In [46]:
#Test with EDF files
def read_data_test(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [47]:
#Test with SET files
def read_data_test_set(file_path):
    data=mne.io.read_raw_eeglab(file_path,preload=True)
    data.set_eeg_reference()
    #data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [48]:
#Test with SET files with selected channels
def read_data_test_af(file_path):
    data=mne.io.read_raw_eeglab(file_path,preload=True)
    data.set_eeg_reference()
    data = data.pick_channels(eeg_channels)
    #data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [49]:
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [50]:
def predict_sc(feature):
    result=[]
    loaded_model = joblib.load('models/LR_SC.sav')
    result.append(test_dict.get(np.bincount(loaded_model.predict(feature)).argmax()))
    loaded_model = joblib.load('models/LDA_Kfold_SC.sav')
    result.append(test_dict.get(np.bincount(loaded_model.predict(feature)).argmax()))
    loaded_model = joblib.load('models/XGboost_SC.sav')
    result.append(test_dict.get(np.bincount(loaded_model.predict(feature)).argmax()))
    return most_frequent(result)

In [51]:
def predict_af(feature):
    result=[]
    loaded_model = joblib.load('models/LDA_Kfold_AF.sav')
    result.append(test_dict_af.get(np.bincount(loaded_model.predict(feature)).argmax()))
    loaded_model = joblib.load('models/XGboost_AF.sav')
    result.append(test_dict_af.get(np.bincount(loaded_model.predict(feature)).argmax()))
    loaded_model = joblib.load('models/RF_Kfold_AF.sav')
    result.append(test_dict_af.get(np.bincount(loaded_model.predict(feature)).argmax()))
    return most_frequent(result)

In [52]:
from scipy import stats

def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [53]:
def load_data(test_file_path):
    extension = os.path.splitext(test_file_path)
    
    if extension[1]=='.edf':
        sample_data_pred=read_data_test(test_file_path)
    else :
        sample_data_pred=read_data_test_set(test_file_path)
    return sample_data_pred

In [54]:
def load_data_af(test_file_path):
    sample_data_pred=read_data_test_af(test_file_path)
    return sample_data_pred

In [55]:
def gen_csv(test_file_path):
    from tqdm.notebook import tqdm
    featuress=[]
    for data in tqdm(load_data(test_file_path)):
        featuress.append(concatenate_features(data))
    featuress=np.array(featuress)
    print(featuress.shape)
    result=predict_sc(featuress)
    if test_dict.get(1)==result:
        print('\nThe subject is : '+ result +'\n')
    else :
        featuress=[]
        for data in tqdm(load_data_af(test_file_path)):
            featuress.append(concatenate_features(data))
        featuress=np.array(featuress)
        print(featuress.shape)
        result=predict_af(featuress)
        print('\nThe subject is having : '+ result +'\n')  
    return result

In [56]:
test_file_path=glob('input/*')
print(len(test_file_path))

26


In [57]:
import csv

start = time.time()
# Heading of csv
data = [
    ['File Path', 'Result']  
    
]

for i in test_file_path:
    # Extract the filename with extension
    filename_with_extension = os.path.basename(i)

    # Split the filename and extension
    filename, extension = os.path.splitext(filename_with_extension)
    result=gen_csv(i)
    data.append([filename,result])
    
# File path
file_path = 'output/result.csv'

# Open the CSV file in write mode
with open(file_path, 'w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)
    
    # Write the data to the CSV file
    csv_writer.writerows(data)

print(f"CSV file '{file_path}' has been generated.")

end = time.time()
print("\nElapsed time :",
      (end-start) * 10**3, "ms")

Extracting EDF parameters from C:\Users\91904\Documents\edf_files_process\test_project\input\h02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
227 matching events found
No baseline correction applied
0 p

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished


  0%|          | 0/227 [00:00<?, ?it/s]

(227, 247)

The subject is : Healthy

Extracting EDF parameters from C:\Users\91904\Documents\edf_files_process\test_project\input\h03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
227 matching events fo

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished


...

  0%|          | 0/154 [00:00<?, ?it/s]

(154, 247)
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
154 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 154 events and 2500 original time points ...
0 bad epochs dropped


C:\Users\91904\AppData\Local\Temp\ipykernel_7992\1605023462.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  data=mne.io.read_raw_eeglab(file_path,preload=True)


  0%|          | 0/154 [00:00<?, ?it/s]

(154, 117)

The subject is having : Dementia

CSV file 'output/result.csv' has been generated.

Elapsed time : 24088.215589523315 ms
